In [38]:
import math
from dataclasses import dataclass
from typing import Union

In [51]:
import math
from dataclasses import dataclass
from typing import Union


@dataclass(frozen=True)
class VRdmaxResult:
    # Inputs
    bw: float
    d: float
    fck: float
    fyk: float
    fywk: float
    θ: float
    αcw: float
    γc: float
    units: str
    # Outputs
    value: float
    z: float
    fcd: float
    v1: float
    tanθ: float
    cotθ: float

    def to_latex(self, show_inputs: bool = True, with_steps: bool = True, decimals: int = 3) -> str:
        """Return a LaTeX string for V_Rd,max formula and results."""
        q = lambda x: f"{x:.{decimals}f}"
        V_unit = "N" if self.units == "N-mm-rad" else "kN"
        L_unit = "mm" if self.units == "N-mm-rad" else "m"
        f_unit = "N/mm^2"

        inputs = (
            "$$" +
            rf"\begin{{array}}{{l l}}"
            rf"b_w = {q(self.bw)}~\mathrm{{{L_unit}}} & d = {q(self.d)}~\mathrm{{{L_unit}}} \\ "
            rf"f_{{ck}} = {q(self.fck)}~\mathrm{{{f_unit}}} & f_{{yk}} = {q(self.fyk)}~\mathrm{{{f_unit}}} \\ "
            rf"f_{{ywk}} = {q(self.fywk)}~\mathrm{{{f_unit}}} & \theta = {q(self.θ)}~\mathrm{{rad}} \\ "
            rf"\alpha_{{cw}} = {q(self.αcw)} & \gamma_c = {q(self.γc)}"
            r"\end{array}$$" 
        ) if show_inputs else ""

        
        # Optional intermediate values
        inter = (
            "$$" +
            rf"\begin{{array}}{{l l}}"
            rf"z = {q(self.z)}~\mathrm{{{L_unit}}} & "
            rf"f_{{cd}} = {q(self.fcd)}~\mathrm{{{f_unit}}} \\ "
            rf"\nu_1 = {q(self.v1)} & "
            rf"\tan\theta = {q(self.tanθ)},~\cot\theta = {q(self.cotθ)}"
            r"\end{array}$$" 
        ) if with_steps else ""

        # Core expression
        expr = (
            r"\begin{align}"
            r"V_{Rd,\max} &= \frac{\alpha_{cw} b_w z \nu_1 f_{cd}}{\cot\theta + \tan\theta}\\[3pt]"
            rf"&= {q(self.value)}~\text{{{V_unit}}}"
            r"\end{align}"
        )
        return "\n".join([x for x in [inputs, inter, expr] if x])

In [52]:
def VRdmax(
    bw: float,
    d: float,
    fck: float,
    fyk: float,
    fywk: float,
    θ: float,
    αcw: float = 1.0,
    γc: float = 1.5,
    units: str = "N-mm-rad",
    include_intermediates: bool = False,
) -> float | VRdmaxResult:
    """Compute V_Rd,max according to Eurocode 2."""
    if units not in ("N-mm-rad", "kN-m-rad"):
        raise ValueError("units must be 'N-mm-rad' or 'kN-m-rad'")

    # Normalize units
    sL = 1000.0 if units == "kN-m-rad" else 1.0
    sF = 0.001 if units == "kN-m-rad" else 1.0
    bw_, d_ = bw * sL, d * sL
    fck_, fyk_, fywk_ = fck * sF, fyk * sF, fywk * sF

    # Core calculations
    z = 0.9 * d_
    fcd = fck_ / γc
    v1 = (0.6 if fywk_ < 0.8 * fyk_ else 0.6 * (1 - fck_ / 250.0))
    tanθ, cotθ = math.tan(θ), 1 / math.tan(θ)
    value = αcw * bw_ * z * v1 * fcd / (tanθ + cotθ)
    if units == "kN-m-rad":
        value *= 0.001

    res = VRdmaxResult(bw, d, fck, fyk, fywk, θ, αcw, γc, units, value, z, fcd, v1, tanθ, cotθ)
    return res if include_intermediates else res.value

In [54]:
import numpy as np

# Just the final value
V = VRdmax(250., 539., 20., 500., 500., np.pi/4)
print(V)  # -> numeric

# Full intermediate result object
res = VRdmax(250., 539., 20., 500., 500., np.pi/4, include_intermediates=True)
print(res.value)

# Generate LaTeX output (for a report)
print(res.to_latex(show_inputs=True, with_steps=True))


446292.00000000006
446292.00000000006
$$\begin{array}{l l}b_w = 250.000~\mathrm{mm} & d = 539.000~\mathrm{mm} \\ f_{ck} = 20.000~\mathrm{N/mm^2} & f_{yk} = 500.000~\mathrm{N/mm^2} \\ f_{ywk} = 500.000~\mathrm{N/mm^2} & \theta = 0.785~\mathrm{rad} \\ \alpha_{cw} = 1.000 & \gamma_c = 1.500\end{array}$$
$$\begin{array}{l l}z = 485.100~\mathrm{mm} & f_{cd} = 13.333~\mathrm{N/mm^2} \\ \nu_1 = 0.552 & \tan\theta = 1.000,~\cot\theta = 1.000\end{array}$$
\begin{align}V_{Rd,\max} &= \frac{\alpha_{cw} b_w z \nu_1 f_{cd}}{\cot\theta + \tan\theta}\\[3pt]&= 446292.000~\text{N}\end{align}


\begin{align}V_{Rd,\max} &= \frac{\alpha_{cw} b_w z \nu_1 f_{cd}}{\cot\theta + \tan\theta}\\[4pt]&= \frac{\alpha_{cw}\,b_w\,z\,\nu_1\,f_{cd}}{1.000+1.000} = 446292.000~\text{N}\end{align}

In [60]:
res = VRdmax(250., 539., 20., 500., 500., np.pi/4, include_intermediates=True)
print(res.d)

539.0


In [56]:
from IPython.display import display, Math, Latex

In [59]:
Latex(res.to_latex(show_inputs=True, with_steps=True))

<IPython.core.display.Latex object>